In [ ]:
# Only run this cell when in Google Colab
! git init
! git remote add origin https://github.com/hannamykula/ob-mts.git
! git fetch
! git checkout -t origin/main

In [ ]:
! pip install tslearn
! pip install scikit-multiflow

In [98]:
from src.process import read_and_split_data, subsample_features
from src.model import train_candidates, cluster_predictions, compute_cluster_representatives, root_mean_square_error, plot_clustering, get_best_num_of_clusters, save_validation_predictions
from src.drift import PageHinkley, DataDrift
from src.predict import predict_n_steps_for_ensemble, predict_one_step_for_ensemble, get_weights, final_prediction_ensemble
import os
import pandas as pd
from config import EXPERIMENT_NAME, VALIDATION_WINDOW_SIZE, WEIGHTS_WINDOW_SIZE, EVALUATION_WINDOW, TARGET_INDEX, SUBSET_SIZE, K, MODEL

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
! wget -q --show-progress -P data/{EXPERIMENT_NAME}/ https://www.dropbox.com/s/ga07hldz2rizkuu/NEW-DATA-1.T15-PREPROCESSED.csv

In [99]:
train, val, test = read_and_split_data(EXPERIMENT_NAME + '/NEW-DATA-1.T15-PREPROCESSED.csv', val_size=VALIDATION_WINDOW_SIZE)

In [100]:
# Hyperparameters
target_index = TARGET_INDEX
subset_size = SUBSET_SIZE
k = K
model = MODEL

In [101]:
sample_subsets = subsample_features(train, target_index, subset_size, k)

In [102]:
train_candidates(train, val, target_index, sample_subsets, model)

Epoch 1/30
68/68 [==============================] - 4s 6ms/step - loss: 172.2187 - accuracy: 0.0000e+00
Epoch 2/30
68/68 [==============================] - 0s 6ms/step - loss: 90.2379 - accuracy: 0.0000e+00
Epoch 3/30
68/68 [==============================] - 0s 5ms/step - loss: 62.8828 - accuracy: 0.0000e+00
Epoch 4/30
68/68 [==============================] - 0s 6ms/step - loss: 51.0645 - accuracy: 0.0000e+00
Epoch 5/30
68/68 [==============================] - 0s 6ms/step - loss: 42.8751 - accuracy: 0.0000e+00
Epoch 6/30
68/68 [==============================] - 0s 6ms/step - loss: 36.6559 - accuracy: 0.0000e+00
Epoch 7/30
68/68 [==============================] - 0s 6ms/step - loss: 31.8423 - accuracy: 0.0000e+00
Epoch 8/30
68/68 [==============================] - 0s 6ms/step - loss: 28.0914 - accuracy: 0.0000e+00
Epoch 9/30
68/68 [==============================] - 0s 6ms/step - loss: 25.2010 - accuracy: 0.0000e+00
Epoch 10/30
68/68 [==============================] - 0s 5ms/step - loss:

In [103]:
VALIDATION_FILENAME = os.path.join(os.getcwd(), 'experiments', EXPERIMENT_NAME, 'validation_predictions_init.csv')

In [104]:
validation_pred = pd.read_csv(VALIDATION_FILENAME)

In [106]:
# num_clusters = 30
num_clusters = get_best_num_of_clusters(validation_pred.transpose(), range(2,20))
cluster_result, cluster_centers = cluster_predictions(validation_pred.transpose(), num_clusters)
ensemble = compute_cluster_representatives(cluster_result, cluster_centers, validation_pred)

Determined number of clusters: 5.0


In [107]:
ph = PageHinkley(delta=0.005, threshold=0.025)
hoeffding = DataDrift(threshold=0.97)

In [108]:
all_model_indices = list(map(str, list(range(1, len(sample_subsets)+1))))

In [109]:
def compute_ensemble(X, running_version_name):
    val_predictions = predict_n_steps_for_ensemble(all_model_indices, X, sample_subsets)
    val_predictions = pd.DataFrame(val_predictions)
    val_predictions.columns = range(1, len(val_predictions.columns)+1)

    cluster_result, cluster_centers = cluster_predictions(val_predictions, num_clusters)
    transpose = val_predictions.transpose()
    transpose.columns = range(1, len(transpose.columns)+1)
    ensemble = compute_cluster_representatives(cluster_result, cluster_centers, transpose)

    return list(map(str, ensemble))

In [110]:
validation = pd.concat([val, test], axis=0)

In [111]:
start_index = 0
test_predictions = []
evaluation_sliding_window_start = 0
version = 1
while start_index < (len(test) - WEIGHTS_WINDOW_SIZE):
    end_index = start_index + WEIGHTS_WINDOW_SIZE
    X_window = test.iloc[start_index:end_index, ]
    y_window = test.iloc[start_index:end_index, target_index]
    X = test.iloc[end_index, ]
    print(start_index)
    weights = get_weights(ensemble, X_window, y_window, sample_subsets)
    pred_at_t = final_prediction_ensemble(ensemble, X, sample_subsets, weights)
    test_predictions.append(pred_at_t)
    if(len(test_predictions) >= EVALUATION_WINDOW):
        X = test.iloc[WEIGHTS_WINDOW_SIZE+evaluation_sliding_window_start:WEIGHTS_WINDOW_SIZE+evaluation_sliding_window_start+EVALUATION_WINDOW, test.columns!=test.columns[target_index]]
        y = test.iloc[WEIGHTS_WINDOW_SIZE+evaluation_sliding_window_start:WEIGHTS_WINDOW_SIZE+evaluation_sliding_window_start+EVALUATION_WINDOW, target_index]
        y_predicted = test_predictions[evaluation_sliding_window_start:evaluation_sliding_window_start+EVALUATION_WINDOW]
        error = root_mean_square_error(y.to_numpy(), y_predicted)
        print(f'Error for window [{WEIGHTS_WINDOW_SIZE+evaluation_sliding_window_start}:{WEIGHTS_WINDOW_SIZE+evaluation_sliding_window_start+EVALUATION_WINDOW}] is {error}')
        ph.add_element(error)
        print(f'Difference: {ph.sum - ph.minimum}')

        glob_min = hoeffding.min
        hoeffding.add_element(X, y)
        print(f'Hoeffding global minimum: {glob_min}. Latest minimum: {hoeffding.min2}. Sample count: {hoeffding.sample_count}')
        if ph.detected_change():
            print('Change in error detected.')
        if hoeffding.detected_change():
            print('Change in data detected')
        if ph.detected_change() | hoeffding.detected_change():
            X_val = validation.iloc[end_index:end_index + VALIDATION_WINDOW_SIZE, ]
            ensemble = compute_ensemble(X_val, version)
            version += 1
        evaluation_sliding_window_start += 1
    start_index = start_index + 1

0
1/1 [==============================] - 0s 494ms/step
1
1/1 [==============================] - 0s 487ms/step
2
1/1 [==============================] - 1s 732ms/step
3
1/1 [==============================] - 0s 493ms/step
4
1/1 [==============================] - 1s 505ms/step
5
1/1 [==============================] - 1s 526ms/step
6
1/1 [==============================] - 1s 515ms/step
7
1/1 [==============================] - 1s 556ms/step
8
1/1 [==============================] - 1s 525ms/step
9
1/1 [==============================] - 0s 478ms/step
10
1/1 [==============================] - 1s 505ms/step
11
1/1 [==============================] - 0s 489ms/step
12
1/1 [==============================] - 1s 608ms/step
13
1/1 [==============================] - 0s 480ms/step
14
1/1 [==============================] - 0s 481ms/step
15
1/1 [==============================] - 0s 497ms/step
16
1/1 [==============================] - 1s 784ms/step
17
1/1 [==============================] - 1s 545ms/step
18

In [112]:
root_mean_square_error(test.iloc[WEIGHTS_WINDOW_SIZE:, target_index].to_numpy(), test_predictions)

0.04351937743397409

In [113]:
# from google.colab import drive
# drive.mount('/content/drive')

save_name = EXPERIMENT_NAME + '_' + MODEL + '_k' + str(K) + '_results.csv'
# path = F'/content/drive/MyDrive/Colab Notebooks/oafbs_results/{save_name}'
path = os.path.join('results\sml10-dataset', save_name)

pd.DataFrame(test_predictions).to_csv(path)